In [ ]:
# python 3.6
# conda

In [3]:
import xml.etree.ElementTree as ET
import pickle
import os
from os import listdir, getcwd
from os.path import join
from PIL import Image

In [2]:
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [ ]:
############################ PUBLIC DATABASE ############################
#http://www.lara.prd.fr/benchmarks/trafficlightsrecognition 
#(free of use for any research purpose)
#
#-----------------------------------------------------------------------#
#Database provided by the *Robotics Centre, of Mines ParisTech* (France)
#
#File format is as follows:
#Timestamp / frameindex x1 y1 x2 y2 id 'type' 'subtype'
#-----------------------------------------------------------------------#
#
#
#File version v 0.5
#['06:11.4062', '/', '4442', '571', '68', '584', '101', '5', 'Traffic', 'Light', 'stop']
# size: width, height
#        640, 480

In [ ]:
labels = set()
count = {}
with open('lara_GT.txt') as gt_file:
    for _, line in enumerate(gt_file):
        split = line.split()
        if (len(split) < 11):
            continue
        # print(split)
        id = split[2]
        
        if (id in count):
            count[id] = count[id] + 1
        else:
            count[id] = 1
        labels.add(split[10])

for id in count:
    if (count[id] > 0):
        print(id.zfill(5))

In [21]:
classes = {'stop': 0, 'go': 1, 'warning': 2}
print(classes)

{'stop': 0, 'go': 1, 'warning': 2}


In [28]:
base = '/media/yu/Data/lara_data/'
image_paths = set()
maps = {}
w = 640
h = 480
with open('lara_GT.txt') as gt_file:
    for _, line in enumerate(gt_file):
        line = line.split()
        label = line[-1]
        if (label not in classes):
            continue
        cls_id = classes[label]
        id = line[2].zfill(6)
        xmin = int(line[3])
        ymin = int(line[4])
        xmax = int(line[5])
        ymax = int(line[6])
        image_file_name = 'frame_' + id
        image_path = base + 'images/' + image_file_name + '.jpg'
        image_paths.add(image_path)
        label_path = base + 'labels/' + image_file_name + '.txt'
        b = (float(xmin), float(xmax), float(ymin), float(ymax))
        bb = convert((w,h), b)
        if (label_path not in maps):
            maps[label_path] = []
        maps[label_path].append(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

In [29]:
with open('train.txt', 'w') as train_file:
    for p in image_paths:
        train_file.write(p + '\n')

In [30]:
for label_path in maps:
    with open(label_path, 'w') as label_file:
        for bb in maps[label_path]:
            label_file.write(bb)